In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("mid").getOrCreate()
sc = spark.sparkContext

22/05/05 21:04:24 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.108.128 instead (on interface ens33)
22/05/05 21:04:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/05 21:04:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import random

### 1

In [6]:
p = min_max_sum_count_map(2)
p

(2, 1)

In [12]:
r = reducer(pair1=(2,1), pair2=(3, 1))
r

(3, 2, 5, 2)

In [23]:
def max_min_sum_count(partition):
    is_first = True
    sum_ = 0
    count = 0
    for element in partition:
        if(is_first==True):
            max_ = element
            min_ = element
            is_first = False
        else:
            max_ = max(element, max_)
            min_ = min(element, min)
        sum_ += element
        count += 1
    return [(max_, min_, sum_, count)]

def min_max_sum_count_map(number):
    return (number, number, number, 1)

def reducer(pair1, pair2):
    return (
    max(pair1[0], pair2[0]), 
    min(pair1[0], pair2[0]), 
    (pair1[0] + pair2[0]), 
    (pair1[1] + pair2[1])
    )

l = [random.randint(1, 1000) for i in range(100)]
rdd1 = sc.parallelize(l)
final = rdd1.map(min_max_sum_count_map).reduce(lambda a,b: (max(a[0], b[0]), min(a[1],b[1]), (a[2]+b[2]), (a[3] + b[3])))
final
#final = rdd.mapPartitions(max_min_sum_count).reduce(lambda a, b: (max(a[0], b[0]), min(a[1], b[1]), sum(a[2], b[2]), sum(a[3], b[3])))
#print("max:" final[0])
#print("min:" final[1])
#print("min:" final[2]/final[3])

(994, 3, 47260, 100)

### 2

In [3]:
def zpn(partition):
    z, p, n = 0, 0, 0
    for element in partition:
        if(element>0):
            p+=1
        elif(element <0):
            n+=1
        else:
            z+=1
    return [(z,p,n)]
    

### 3 ?

In [5]:
'''
def map(key, value):
    emit(key, (value, 1))

def reduce(key, value): key->key (value, 1) -> value
    emit(sum(value[0] / sum(value[1])))
'''

'\ndef map(key, value):\n    emit(key, (value, 1))\n\ndef reduce(key, value): key->key (value, 1) -> value\n    emit(sum(value[0] / sum(value[1])))\n'

### 4

In [ ]:
'''
distinct()

union()
'''

### 5

In [ ]:
'''

'''

### 6

In [ ]:
'''
joined frame:

b1, c1, NaN
b1, c2, NaN
b2, a2, NaN
b7, c1, a1
b7, c2, a2


def map(record):
    rec = record.split(',')
    key = rec[1]
    value = rec[2]
    emit(key, value)

def reduce(key: 'b', values: 'a' or 'c'):
    


'''

### 8

In [4]:
id_Grades = []
import random
for i in range(100):
    id_Grades.append(str(random.randint(1, 10)) + ',' + str(random.randint(1, 100)))

In [7]:

def mapper(record):
    rec = record.split(',')
    name = rec[0]
    grade = int(rec[1])

    return (name, grade)

def select_min_max(values):
    return (min(values), max(values))


rdd8 = sc.parallelize(id_Grades)
result = rdd8.map(mapper).groupByKey().mapValues(select_min_max).collect()
result

[('4', (8, 90)),
 ('9', (50, 100)),
 ('10', (14, 100)),
 ('1', (5, 92)),
 ('8', (3, 97)),
 ('2', (5, 89)),
 ('5', (1, 96)),
 ('3', (13, 92)),
 ('7', (10, 79)),
 ('6', (2, 88))]

### 11

In [14]:
rdd11 = sc.textFile('11.txt')
def mapper(record):
    words = record.split(',')
    pairs_list = []
    for i in range(len(words) - 1):
        pairs_list.append((words[i], words[i+1]))
    return pairs_list
#rdd11.collect()
tmp = rdd11.flatMap(mapper).map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b).collect()
tmp

[(('you', 'doing'), 2),
 (('Hello', 'how'), 3),
 (('how', 'are'), 4),
 (('are', 'you'), 4),
 (('Hi', 'how'), 1)]

### 12

#### a

In [16]:
'''
("k1", 4)
(k2, 2)
(k3, 2)
(k1, 2)
(k2, 2)
(k2, 2)
(k3, 8)
(k3, 6)
'''

'\n(k1, 4)\n(k2, 2)\n(k3, 2)\n(k1, 2)\n(k2, 2)\n(k2, 2)\n(k3, 8)\n(k3, 6)\n'

#### b

In [17]:
'''
"k1" : [4, 2]
"k2" : [2, 2, 2]
"k3" : [2, 8, 6]
'''

'\n"k1" : [4, 2]\n"k2" : [2, 2, 2]\n"k3" : [2, 8, 6]\n'

#### c

In [18]:
'''
(("k1", 6), ("k2", 6), ("k3", 16))
'''

'\n(("k1", 6), ("k2", 6), ("k3", 16))\n'

### 13 ?

### 14

In [36]:
rdd14 = sc.textFile("16")
#rdd14.getNumPartitions()
#print(rdd14.collect())
def mapper(sentence):
    words = sentence.split(' ')
    cnt = 0
    pcnt = 0
    for word in words:
        cnt+=1
        length = len(word)-1
        is_p = True
        for i in range(len(word)):
            if(word[i] != word[length-i]):
                is_p=False
                break
        if(is_p == True):
            pcnt+=1
    return (pcnt, cnt)

def reducer(pair1, pair2):
    return (pair1[0]+pair2[0], pair1[1] + pair2[1])

#pair = mapper('wow i thought i did forgot all the stuff you told me')
#pair
result = rdd14.map(mapper).reduce(lambda a,b: (a[0]+b[0], a[1]+b[1]))
print(result[0]/result[1])


0.3333333333333333


### 17?


### 18
+ a: format of the output of map: `("key", value),("key", key), ("key1",1)/("key2", 2)`
+ b: ("key", []) ("key1", []) ("key2", [])
+ c: 3
+ d: pass


### 19

In [ ]:
'''
def mapper(key: documentNumber, value: document):
    words = document.split(' ')
    for i in range(len(words)):
        emit(words[i], (key, i))

def reducer(key:words[i], (documentNumber, i)):
    for 
'''